# Cleaning Data with PySpark - Part 3

## Improving Performance
Improve data cleaning tasks by increasing performance or reducing resource requirements.

In [1]:
BUCKET = 'driven-actor-210609'

In [6]:
import time

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 12:17:25 INFO SparkEnv: Registering MapOutputTracker
25/03/13 12:17:26 INFO SparkEnv: Registering BlockManagerMaster
25/03/13 12:17:26 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/03/13 12:17:26 INFO SparkEnv: Registering OutputCommitCoordinator


### Caching a DataFrame
You've been assigned a task that requires running several analysis operations on a DataFrame. You've learned that caching can improve performance when reusing DataFrames and would like to implement it.

You'll be working with a new dataset consisting of airline departure information. It may have repetitive data and will need to be de-duplicated.

The DataFrame `departures_df` is defined, but no actions have been performed.

In [4]:
file_path = f'gs://{BUCKET}/pyspark/datasets/AA_DFW_2017_Departures.csv.gz'
# file_path = 'datasets/AA_DFW_2017_Departures.csv.gz'

departures_df = spark.read.format('csv').options(Header=True).load(file_path)

In [7]:
start_time = time.time()

# Add caching to the unique rows in departures_df
departures_df = departures_df.distinct().cache()

# Count the unique rows in departures_df, noting how long the operation takes
print("Counting %d rows took %f seconds" % (departures_df.count(), time.time() - start_time))

# Count the rows again, noting the variance in time of a cached DataFrame
start_time = time.time()
print("Counting %d rows again took %f seconds" % (departures_df.count(), time.time() - start_time))

Counting 139358 rows took 24.519457 seconds


Counting 139358 rows again took 5.385797 seconds


### Removing a DataFrame from cache
You've finished the analysis tasks with the departures_df DataFrame, but have some other processing to do. You'd like to remove the DataFrame from the cache to prevent any excess memory usage on your cluster.

The DataFrame `departures_df` is defined and has already been cached for you.

In [8]:
# Determine if departures_df is in the cache
print("Is departures_df cached?: %s" % departures_df.is_cached)
print("Removing departures_df from cache")

# Remove departures_df from the cache
departures_df.unpersist()

# Check the cache status again
print("Is departures_df cached?: %s" % departures_df.is_cached)

Is departures_df cached?: True
Removing departures_df from cache
Is departures_df cached?: False


### Object splitting

In [ ]:
# Split large file departures_full.txt to files with 10000 rows per file and chunk prefix. e.g: chunk-00, chunk-01, chunk-02, ...
!split -l 10000 -d departures_full.txt chunk-

In [ ]:
# Write large file as parquet data format to improve performance
df_csv = spark.read.csv('departures_full.txt.gz')
df_csv.write.parquet('departures.parquet')
df = spark.read.parquet('departures.parquet')

### File import performance
You've been given a large set of data to import into a Spark DataFrame. You'd like to test the difference in import speed by splitting up the file.

You have two types of files available: `departures_full.txt.gz` and `departures_xxx.txt.gz` where xxx is 000 - 013. The same number of rows is split between each file.

In [10]:
full_file_path = f'gs://{BUCKET}/pyspark/datasets/AA_DFW_2017_Departures.csv.gz'
split_file_path = f'gs://{BUCKET}/pyspark/datasets/AA_DFW_2017_Departures_???.csv.gz'

In [11]:
# Import the full and split files into DataFrames
full_df = spark.read.csv(full_file_path)
split_df = spark.read.csv(split_file_path)

# Print the count and run time for each DataFrame
start_time_a = time.time()
print("Total rows in full DataFrame:\t%d" % full_df.count())
print("Time to run: %f" % (time.time() - start_time_a))

start_time_b = time.time()
print("Total rows in split DataFrame:\t%d" % split_df.count())
print("Time to run: %f" % (time.time() - start_time_b))

Total rows in full DataFrame:	139359
Time to run: 0.606498


Total rows in split DataFrame:	139359
Time to run: 0.961519


### Reading Spark configurations
You've recently configured a cluster via a cloud provider. Your only access is via the command shell or your python code. You'd like to verify some Spark settings to validate the configuration of the cluster.

The `spark` object is available for use.

In [12]:
# Name of the Spark application instance
app_name = spark.conf.get('spark.app.name')

# Driver TCP port
driver_tcp_port = spark.conf.get('spark.driver.port')

# Number of join partitions
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')

# Show the results
print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions)

Name: pyspark-shell
Driver TCP port: 35173
Number of partitions: 1000


### Writing Spark configurations
Now that you've reviewed some of the Spark configurations on your cluster, you want to modify some of the settings to tune Spark to your needs. You'll import some data to review that your changes have affected the cluster.

The spark configuration is initially set to the default value of 200 partitions.

The `spark` object is available for use. A file named `departures.txt.gz` is available for import. An initial DataFrame containing the distinct rows from `departures.txt.gz` is available as `departures_df`.

In [ ]:
file_path = f'gs://{BUCKET}/pyspark/datasets/AA_DFW_2017_Departures.csv.gz'

In [24]:
departures_df = spark.read.csv(file_path).distinct()

# Store the number of partitions in variable
before = departures_df.rdd.getNumPartitions()

# Configure Spark to use 500 partitions
spark.conf.set('spark.sql.shuffle.partitions', 10)

# Recreate the DataFrame using the departures data file
departures_df = spark.read.csv(file_path).distinct()

# Print the number of partitions for each instance
print("Partition count before change: %d" % before)
print("Partition count after change: %d" % departures_df.rdd.getNumPartitions())

Partition count before change: 2
Partition count after change: 1


### Normal joins
You've been given two DataFrames to combine into a single useful DataFrame. Your first task is to combine the DataFrames normally and view the execution plan.

The DataFrames `flights_df` and `airports_df` are available to you.

In [36]:
flights_file_path = f'gs://{BUCKET}/pyspark/datasets/AA_DFW_2018_Departures.csv.gz'
flights_df = spark.read.options(Header=True).csv(flights_file_path)
flights_df.show(10, truncate=False)

+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|01/01/2018       |0005         |HNL                |498                          |
|01/01/2018       |0007         |OGG                |501                          |
|01/01/2018       |0043         |DTW                |0                            |
|01/01/2018       |0051         |STL                |100                          |
|01/01/2018       |0075         |DCA                |147                          |
|01/01/2018       |0096         |STL                |92                           |
|01/01/2018       |0103         |SJC                |227                          |
|01/01/2018       |0119         |OGG                |517                          |
|01/01/2018       |0123         |HNL                |489                    

In [37]:
airports_file_path = f'gs://{BUCKET}/pyspark/datasets/airportnames.txt.gz'
airports_df = spark.read.options(Header=True).csv(airports_file_path)
airports_df.show(10, truncate=False)

+-------------------------------------------+----+
|AIRPORTNAME                                |IATA|
+-------------------------------------------+----+
|Goroka Airport                             |GKA |
|Madang Airport                             |MAG |
|Mount Hagen Kagamuga Airport               |HGU |
|Nadzab Airport                             |LAE |
|Port Moresby Jacksons International Airport|POM |
|Wewak International Airport                |WWK |
|Narsarsuaq Airport                         |UAK |
|Godthaab / Nuuk Airport                    |GOH |
|Kangerlussuaq Airport                      |SFJ |
|Thule Air Base                             |THU |
+-------------------------------------------+----+
only showing top 10 rows



In [44]:
# Join the flights_df and aiports_df DataFrames
normal_df = flights_df.join(airports_df, \
    flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan
normal_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [Destination Airport#874], [IATA#920], Inner, BuildRight, false
   :- Filter isnotnull(Destination Airport#874)
   :  +- FileScan csv [Date (MM/DD/YYYY)#872,Flight Number#873,Destination Airport#874,Actual elapsed time (Minutes)#875] Batched: false, DataFilters: [isnotnull(Destination Airport#874)], Format: CSV, Location: InMemoryFileIndex(1 paths)[gs://driven-actor-210609/pyspark/datasets/AA_DFW_2018_Departures.csv.gz], PartitionFilters: [], PushedFilters: [IsNotNull(Destination Airport)], ReadSchema: struct<Date (MM/DD/YYYY):string,Flight Number:string,Destination Airport:string,Actual elapsed ti...
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, false]),false), [plan_id=1461]
      +- Filter isnotnull(IATA#920)
         +- FileScan csv [AIRPORTNAME#919,IATA#920] Batched: false, DataFilters: [isnotnull(IATA#920)], Format: CSV, Location: InMemoryFileIndex(1 paths)[gs://driven-actor-2

### Using broadcasting on Spark joins
Remember that table joins in Spark are split between the cluster workers. If the data is not local, various shuffle operations are required and can have a negative impact on performance. Instead, we're going to use Spark's `broadcast` operations to give each node a copy of the specified data.

A couple tips:
- Broadcast the smaller DataFrame. The larger the DataFrame, the more time required to transfer to the worker nodes.
- On small DataFrames, it may be better skip broadcasting and let Spark figure out any optimization on its own.
- If you look at the query execution plan, a broadcastHashJoin indicates you've successfully configured broadcasting.

The DataFrames `flights_df` and `airports_df` are available to you.

In [45]:
# Import the broadcast method from pyspark.sql.functions
from pyspark.sql.functions import broadcast

# Join the flights_df and airports_df DataFrames using broadcasting
broadcast_df = flights_df.join(broadcast(airports_df), \
    flights_df["Destination Airport"] == airports_df["IATA"] )

# Show the query plan and compare against the original
broadcast_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [Destination Airport#874], [IATA#920], Inner, BuildRight, false
   :- Filter isnotnull(Destination Airport#874)
   :  +- FileScan csv [Date (MM/DD/YYYY)#872,Flight Number#873,Destination Airport#874,Actual elapsed time (Minutes)#875] Batched: false, DataFilters: [isnotnull(Destination Airport#874)], Format: CSV, Location: InMemoryFileIndex(1 paths)[gs://driven-actor-210609/pyspark/datasets/AA_DFW_2018_Departures.csv.gz], PartitionFilters: [], PushedFilters: [IsNotNull(Destination Airport)], ReadSchema: struct<Date (MM/DD/YYYY):string,Flight Number:string,Destination Airport:string,Actual elapsed ti...
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, false]),false), [plan_id=1484]
      +- Filter isnotnull(IATA#920)
         +- FileScan csv [AIRPORTNAME#919,IATA#920] Batched: false, DataFilters: [isnotnull(IATA#920)], Format: CSV, Location: InMemoryFileIndex(1 paths)[gs://driven-actor-2

### Comparing broadcast vs normal joins
You've created two types of joins, normal and broadcasted. Now your manager would like to know what the performance improvement is by using Spark optimizations. If the results are promising, you'll be given more opportunity to tweak the Spark setup as needed.

Your DataFrames `normal_df` and `broadcast_df` are available for your use.

In [ ]:
start_time = time.time()
# Count the number of rows in the normal DataFrame
normal_count = normal_df.count()
normal_duration = time.time() - start_time

start_time = time.time()
# Count the number of rows in the broadcast DataFrame
broadcast_count = broadcast_df.count()
broadcast_duration = time.time() - start_time

# Print the counts and the duration of the tests
print("Normal count:\t\t%d\tduration: %f" % (normal_count, normal_duration))
print("Broadcast count:\t%d\tduration: %f" % (broadcast_count, broadcast_duration))